In [176]:
from davitpy import pydarn
import datetime
import numpy
import pandas

In [133]:
startTime = datetime.datetime(2017,12,2)
endTime = datetime.datetime(2017,12,3)
radSel = 'bks'
fileType = 'fitacf3'
filtered = False

In [66]:
f = pydarn.sdio.radDataOpen(startTime, radSel,
                    endTime, filtered=filtered,
                    fileType=fileType)

In [67]:
myBeam = pydarn.sdio.radDataReadRec(f)
print myBeam.recordDict.keys()

['x_p_l', 'origin.command', 'mplgs', 'txpow', 'nave', 'x_gflg', 'x_sd_phi', 'scan', 'time', 'atten', 'noise.vel', 'smsep', 'stat.lopwr', 'v_e', 'x_v_e', 'x_w_s', 'mplgexs', 'x_p_s', 'radar.revision.minor', 'xcf', 'nlag', 'cp', 'gflg', 'p_s', 'w_s', 'p_l', 'origin.code', 'pwr0', 'stat.agc', 'intt.sc', 'noise.mean', 'intt.us', 'x_w_l_e', 'stid', 'mxpwr', 'slist', 'x_sd_l', 'noise.lag0', 'p_s_e', 'w_l_e', 'x_sd_s', 'rsep', 'p_l_e', 'mppul', 'ercod', 'fitacf.revision.minor', 'radar.revision.major', 'channel', 'ltab', 'elv_high', 'frang', 'bmazm', 'x_qflg', 'noise.sky', 'x_w_s_e', 'fitacf.revision.major', 'lagfr', 'x_v', 'lvmax', 'sd_l', 'w_s_e', 'x_p_l_e', 'phi0', 'ifmode', 'offset', 'sd_s', 'bmnum', 'x_p_s_e', 'elv_low', 'noise.search', 'tfreq', 'ptab', 'txpl', 'elv', 'origin.time', 'x_w_l', 'phi0_e', 'w_l', 'rxrise', 'combf', 'v', 'mpinc', 'qflg', 'nrang', 'sd_phi']


In [68]:
# print myBeam.recordDict["time"]
# print myBeam.recordDict["qflg"]
# print myBeam.recordDict["bmnum"]

In [69]:
mb = pydarn.sdio.radDataReadAll(f)

In [177]:
# we'll create a dataframe with all the data
# To keep things simple (at this moment), I'm
# not using multi-indexed dataframe.
# Initialize empty arrays for the data
# these will later be converted to a DF
qflgArr = numpy.empty([0])
gateArr = numpy.empty([0])
velArr = numpy.empty([0])
spwArr = numpy.empty([0])
pwrArr = numpy.empty([0])
gflgArr = numpy.empty([0])
dtArr = numpy.empty([0])
bmArr = numpy.empty([0])
for n, m in enumerate(mb):
#     print numpy.array(m.fit.qflg )
    if m is None:
        continue
    if m.fit is None:
        continue
    if m.fit.v is None:
        continue
    qflgArr = numpy.concatenate( ( qflgArr, numpy.array(m.fit.qflg ) ) )
    gateArr = numpy.concatenate( ( gateArr, numpy.array(m.fit.slist ) ) )
    velArr = numpy.concatenate( ( velArr, numpy.array(m.fit.v ) ) )
    spwArr = numpy.concatenate( ( spwArr, numpy.array(m.fit.w_l ) ) )
    pwrArr = numpy.concatenate( ( pwrArr, numpy.array(m.fit.p_l ) ) )
    gflgArr = numpy.concatenate( ( gflgArr, numpy.array(m.fit.gflg ) ) )
    # date and bmnum are a little different
    dtArr = numpy.concatenate( ( dtArr, numpy.full( (len(m.fit.qflg)), m.time) ) )
    bmArr = numpy.concatenate( ( bmArr, numpy.full( (len(m.fit.qflg)), m.bmnum) ) )
dfx = pandas.DataFrame( {
    "qflg" : qflgArr,
    "gate" : gateArr,
    "vel" : velArr,
    "spw" : spwArr,
    "pwr" : pwrArr,
    "gflg" : gflgArr,
    "date" : dtArr,
    "beam" : bmArr,
})
# Convert int cols to int16
# they are being stored as 
# floats now! save space!
intCols = [ "qflg", "gate", "gflg", "beam" ]
dfx[intCols] = dfx[intCols].astype(numpy.int16)

In [178]:
dfx.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501574 entries, 0 to 501573
Data columns (total 8 columns):
beam    501574 non-null int16
date    501574 non-null datetime64[ns]
gate    501574 non-null int16
gflg    501574 non-null int16
pwr     501574 non-null float64
qflg    501574 non-null int16
spw     501574 non-null float64
vel     501574 non-null float64
dtypes: datetime64[ns](1), float64(3), int16(4)
memory usage: 19.1 MB


In [179]:
print dfx.head()
print "------------------"
print dfx.dtypes

   beam                    date  gate  gflg        pwr  qflg        spw  \
0    21 2015-01-01 00:00:05.327     1     0   5.017268     1  37.019970   
1    21 2015-01-01 00:00:05.327     2     1   7.522198     1  12.723468   
2    21 2015-01-01 00:00:05.327     3     1   4.680641     1  12.176193   
3    21 2015-01-01 00:00:05.327    24     1  12.528772     1   0.010000   
4    21 2015-01-01 00:00:05.327    25     1  17.955828     1   0.010000   

         vel  
0 -28.414904  
1 -18.902945  
2 -19.732080  
3   4.341410  
4  -3.512275  
------------------
beam             int16
date    datetime64[ns]
gate             int16
gflg             int16
pwr            float64
qflg             int16
spw            float64
vel            float64
dtype: object


In [161]:
print mb[0].fit.qflg
print "---------------------------"
# print mb[0].fit.slist
# print "---------------------------"
# print mb[0].fit.v
# print "---------------------------"
# print mb[0].fit.w_l
# print "---------------------------"
# print mb[0].fit.p_l
# print "---------------------------"
# print mb[0].fit.gflg
# print "---------------------------"
# print mb[0].time
# print "---------------------------"
# print numpy.full( (len(mb[0].fit.qflg)), mb[0].bmnum).shape
print numpy.array(mb[0].fit.slist )

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
---------------------------
[ 1  2  3 24 25 26 27 28 29 30 31 32 33 44]


In [117]:
x = numpy.empty([0])
print x
x = numpy.concatenate( (x,numpy.array(mb[0].fit.v)) )
print x
x = numpy.concatenate( (x,numpy.array(mb[10].fit.v)) )
print x.shape
# print numpy.concatenate( (numpy.array(mb[0].fit.v), numpy.array(mb[10].fit.v)) ).shape

[]
[-28.41490364 -18.90294456 -19.73208046   4.34141016  -3.5122745
  -2.17070508  -3.82897639  -5.99968147  -6.39114666  -3.5122745
  -1.34156954  -0.8291356   -0.8291356    0.        ]
(36,)


In [114]:
print len(mb[0].fit.v + mb[10].fit.v)

14


In [134]:
# f = pydarn.sdio.radDataOpen(startTime, radSel,
#                     endTime, filtered=filtered,
#                     fileType=fileType)
# xx = pydarn.sdio.radDataReadRec(f)
# print xx.fType